# Spark DataFrames Project Exercise 

Let's get some quick practice with your new Spark DataFrame skills, you will be asked some basic questions about some stock market data, in this case Walmart Stock from the years 2012-2017. This exercise will just ask a bunch of questions, unlike the future machine learning exercises, which will be a little looser and be in the form of "Consulting Projects", but more on that later!

For now, just answer the questions and complete the tasks below.

#### Use the walmart_stock.csv file to Answer and complete the  tasks below!

#### Start a simple Spark Session

In [0]:
from pyspark.sql import SparkSession; 
spark = SparkSession.builder.appName("WalmartStockAnalysis").getOrCreate()


#### Load the Walmart Stock CSV File, have Spark infer the data types.

In [0]:
walmart_stock_df = spark.read.csv("/FileStore/tables/Walmart_stock.csv", header=True, inferSchema=True)


#### What are the column names?

In [0]:
walmart_stock_df.columns


Out[4]: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### What does the Schema look like?

In [0]:
walmart_stock_df.printSchema()


root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



#### Print out the first 5 columns.

In [0]:
walmart_stock_df.show(5)


+----------+---------+---------+---------+---------+--------+---------+
|      Date|     Open|     High|      Low|    Close|  Volume|Adj Close|
+----------+---------+---------+---------+---------+--------+---------+
|2012-01-03|59.970001|61.060001|59.869999|60.330002|12668800|52.619235|
|2012-01-04|60.209999|60.349998|59.470001|59.709999| 9593300|52.078475|
|2012-01-05|59.349998|59.619999|58.369999|59.419998|12768200|51.825539|
|2012-01-06|59.419998|59.450001|58.869999|     59.0| 8069400| 51.45922|
|2012-01-09|59.029999|59.549999|58.919998|    59.18| 6679300|51.616215|
+----------+---------+---------+---------+---------+--------+---------+
only showing top 5 rows



#### Use describe() to learn about the DataFrame.

In [0]:
walmart_stock_df.describe().show()


+-------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|             Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|             1258|             1258|             1258|             1258|             1258|             1258|
|   mean|72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev| 6.76809024470826|6.768186808159218|6.744075756255496| 6.75685916373299|  4519780.8431556|6.722609449996858|
|    min|        56.389999|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|        90.800003|        90.970001|            89.25|        90.470001|         80898100|        84.914216|
+-------+-----------------+-----------------+-----------

## Bonus Question!
#### There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar. [Check this link for a hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast)

If you get stuck on this, don't worry, just view the solutions.

In [0]:
from pyspark.sql.functions import format_number; 
walmart_stock_df.describe().select("summary", *[format_number(walmart_stock_df.describe().select(c).alias(c).first()[c], 2).alias(c) 
for c in walmart_stock_df.columns 
if c != "Date"]).show()


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2009930341318589>:2
      1 from pyspark.sql.functions import format_number; 
----> 2 walmart_stock_df.describe().select("summary", *[format_number(walmart_stock_df.describe().select(c).alias(c).first()[c], 2).alias(c) 
      3 for c in walmart_stock_df.columns 
      4 if c != "Date"]).show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:3023, in DataFrame.select(self, *cols)
   2978 def select(self, *cols: "ColumnOrName") -> "DataFrame":  # type

#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [0]:
hv_df = walmart_stock_df.withColumn("HV Ratio", walmart_stock_df["High"] / walmart_stock_df["Volume"])
hv_df.select("Date", "High", "Volume", "HV Ratio").show(5)


+----------+---------+--------+--------------------+
|      Date|     High|  Volume|            HV Ratio|
+----------+---------+--------+--------------------+
|2012-01-03|61.060001|12668800|4.819714653321546E-6|
|2012-01-04|60.349998| 9593300|6.290848613094555E-6|
|2012-01-05|59.619999|12768200|4.669412994783916E-6|
|2012-01-06|59.450001| 8069400|7.367338463826307E-6|
|2012-01-09|59.549999| 6679300|8.915604778943901E-6|
+----------+---------+--------+--------------------+
only showing top 5 rows



#### What day had the Peak High in Price?

In [0]:
walmart_stock_df.orderBy(walmart_stock_df["High"].desc()).select("Date", "High").show(1)


+----------+---------+
|      Date|     High|
+----------+---------+
|2015-01-13|90.970001|
+----------+---------+
only showing top 1 row



#### What is the mean of the Close column?

In [0]:
from pyspark.sql.functions import mean

walmart_stock_df.select(mean("Close")).show()


+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



#### What is the max and min of the Volume column?

In [0]:
from pyspark.sql.functions import max

walmart_stock_df.select(max("Volume")).show()


+-----------+
|max(Volume)|
+-----------+
|   80898100|
+-----------+



In [0]:
from pyspark.sql.functions import min

walmart_stock_df.select(min("Volume")).show()


+-----------+
|min(Volume)|
+-----------+
|    2094900|
+-----------+



#### How many days was the Close lower than 60 dollars?

In [0]:
walmart_stock_df.filter(walmart_stock_df["Close"] < 60).count()


Out[14]: 81

#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [0]:
# Total number of days
total_days = walmart_stock_df.count()

# Number of days High > 80
high_over_80 = walmart_stock_df.filter(walmart_stock_df["High"] > 80).count()

# Percentage
percentage = (high_over_80 / total_days) * 100
print(f"Percentage of days High > 80: {percentage:.2f}%")


Percentage of days High > 80: 9.14%


#### What is the Pearson correlation between High and Volume?
#### [Hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameStatFunctions.corr)

In [0]:
correlation = walmart_stock_df.corr('High', 'Volume')
print(f"Pearson correlation between High and Volume: {correlation}")


Pearson correlation between High and Volume: -0.33843260617371607


#### What is the max High per year?

In [0]:
from pyspark.sql.functions import year, month, max, avg

# For max High per year
walmart_stock_df.select(year("Date").alias("Year"), "High").groupBy("Year").agg(max("High").alias("Max_High")).orderBy("Year").show()


+----+---------+
|Year| Max_High|
+----+---------+
|2012|77.599998|
|2013|81.370003|
|2014|88.089996|
|2015|90.970001|
|2016|75.190002|
+----+---------+



#### What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months. 

In [0]:
from pyspark.sql.functions import year, month, max, avg
# For average Close per calendar month
walmart_stock_df.select(month("Date").alias("Month"), "Close").groupBy("Month").agg(avg("Close").alias("Avg_Close")).orderBy("Month").show()


+-----+-----------------+
|Month|        Avg_Close|
+-----+-----------------+
|    1|71.44801958415842|
|    2|  71.306804443299|
|    3|71.77794377570092|
|    4|72.97361900952382|
|    5|72.30971688679247|
|    6| 72.4953774245283|
|    7|74.43971943925233|
|    8|73.02981855454546|
|    9|72.18411785294116|
|   10|71.57854545454542|
|   11| 72.1110893069307|
|   12|72.84792478301885|
+-----+-----------------+



# Great Job!